In [ ]:
from genepy.google import gcp
import pandas as pd
from depmapomics.tracker import initTracker, SampleTracker
from depmapomics.config_prod import *
from taigapy import TaigaClient
tc = TaigaClient()
%load_ext autoreload
%autoreload 2

In [ ]:
track = SampleTracker(
        my_id=MY_ID,
        mystorage_id=MYSTORAGE_ID,
        sheetcreds=SHEETCREDS,
        refsheet_url=REFSHEET_URL,
        depmap_pv=DEPMAP_PV,
        samples_not_found_url="https://docs.google.com/spreadsheets/d/1yC3brpov3JELvzNoQe3eh0W196tfXzvpa0jUezMAxIg",
        samples_missing_arxspan_url="https://docs.google.com/spreadsheets/d/1htfgpXrMvXDlqbcZltpq6vOE_Vo2YZ3-3mdaXV-Irzk",
        gumbo_url='https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg',
        gumbo_sheetname="Backfilled profile IDs",
    )

## updating the pipeline

In [ ]:
tracker = track.read_tracker()

In [ ]:
rn = {u: v['arxspan_id'] for u,v in tracker[['arxspan_id']].iterrows()}

In [ ]:
import dalmatian as dm
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")

In [ ]:
samp = wm.get_samples()

## making the files

In [ ]:
oldmaf = pd.read_csv(dm.WorkspaceManager('broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline').get_sample_sets().loc['all','filtered_CGA_MAF_aggregated'], sep='\t')

In [ ]:
oldmaf

In [ ]:
res = []
for file in samp[~samp.depmap_maf.isna()].depmap_maf.tolist():
    maf = pd.read_csv(file)
    maf["sample_id"] = file.split('/')[-1].split('-maf-')[0]
    res.append(maf)
res = pd.concat(res)

In [ ]:
oldmaf

In [ ]:
oldmaf = oldmaf.rename(columns={'Chromosome':'chrom', 'Start_position':'start','Tumor_Sample_Barcode': 'sample_id', 'Reference_Allele': 'ref', 'Tumor_Seq_Allele2': 'alt'})

In [ ]:
oldmaf = oldmaf[oldmaf.sample_id.isin(set(res["sample_id"]))]

In [ ]:
res = res[res["sample_id"].isin(set(oldmaf.sample_id))]

In [ ]:
len(set(oldmaf.sample_id)), len(set(res["sample_id"]))

In [ ]:
res.to_csv('data/new.csv', index=False)

In [ ]:
res.iloc[:70000].replace({'sample_id':rn}).to_csv('data/newmaf_subset.csv.gz', index=False)

In [ ]:
res = pd.read_csv('data/new.csv')

In [ ]:
res

## converting

In [ ]:
from liftover import get_lifter
converter = get_lifter('hg19', 'hg38')

In [ ]:
new = []
for (chrom,pos) in oldmaf[['chrom','start']].values:
    d = converter[chrom][pos]
    if len(d)>0:
        new.append(list(d[0])[:2])
    else:
        new.append([chrom,pos])
    
oldmaf[['chrom','start']] = np.array(new)

In [ ]:
print(len(oldmaf))
oldmaf = oldmaf[oldmaf['tumor_f']>=0.15]
print(len(oldmaf))

In [ ]:
oldmaf = oldmaf.drop(columns=['Center',  'NCBI_Build', 'End_position', 'Strand', 'PASS', 'dbSNP_Val_Status','dbSNP_RS', 'Tumor_Seq_Allele1'])

In [ ]:
oldmaf[oldmaf['Variant_Type']=='INS']

In [ ]:
res.loc[res['variant_type']=='DEL', 'alt'] = '-'
res.loc[res['variant_type']=='DEL', 'ref'] = res.loc[res['variant_type']=='DEL', 'ref'].str[1:]
res.loc[res['variant_type']=='DEL', 'pos'] = res.loc[res['variant_type']=='DEL', 'pos']+1

In [ ]:
res.loc[res['variant_type']=='INS', 'ref'] = '-'
res.loc[res['variant_type']=='INS', 'alt'] = res.loc[res['variant_type']=='INS', 'alt'].str[1:]
res.loc[res['variant_type']=='INS', 'pos'] = res.loc[res['variant_type']=='INS', 'pos']+1

In [ ]:
res['locat'] = res.sample_id+":"+res.chrom+":"+res.pos.astype(str)
res['id'] = res.locat + ":"+res.alt

oldmaf['locat'] = oldmaf.sample_id+ ':'+oldmaf.chrom.astype(str) + ":" + oldmaf.start.astype(str)
oldmaf['id'] = oldmaf.locat + ":"+oldmaf.alt

In [ ]:
oldmaf = oldmaf.sort_values(by='locat')
res = res.sort_values(by='locat')

In [ ]:
oldmaf

In [ ]:
res = res.drop_duplicates('locat')
res

## checking overlap

In [ ]:
a = len(set(res.locat) & set(oldmaf.locat))
b = len(set(res.locat) ^ set(oldmaf.locat))
a / (a+b)

In [ ]:
len(set(res.locat)),len(set(oldmaf.locat))

In [ ]:
## twice as less cosmic hotspot on average than what we are supposed to have (cosmic info is not used to filter) !!!

In [ ]:
oldmaf.isCOSMIChotspot.sum() / len(oldmaf)

In [ ]:
oldmaf[oldmaf.locat.isin(set(oldmaf.locat)- set(res.locat))].reset_index().iloc[:].isCOSMIChotspot.sum() / len(oldmaf[oldmaf.locat.isin(set(oldmaf.locat)- set(res.locat))])

In [ ]:
oldmaf[oldmaf.locat.isin(set(oldmaf.locat)- set(res.locat))].reset_index().iloc[181:210]

In [ ]:
1. CDS-00rz9N:chr10:102416887	clustered_event
2. CDS-00rz9N:chr10:68969042	DNP
3. CDS-00rz9N:chr11:105002202	wrong mapping
4. CDS-00rz9N:chr11:120424378	germline
5. CDS-00rz9N:chr11:59212836	germline
6. CDS-00rz9N:chr11:75668254	base_qual
7. CDS-00rz9N:chr12:27780599	conversion
8. CDS-00rz9N:chr12:6748879	conversion
9. CDS-00rz9N:chr13:41131871	germline
10.CDS-00rz9N:chr13:47988897	germline
11.CDS-00rz9N:chr14:20060702	germline
12.CDS-00rz9N:chr14:20401510	germline
13.CDS-00rz9N:chr14:26595549	germline
14.CDS-00rz9N:chr15:22909259	conversion (+2)
15.CDS-00rz9N:chr15:79923022	conversion
16.CDS-00rz9N:chr16:66912890	not detected by M2??A->C
17.CDS-00rz9N:chr16:780252	INTRON, not splice site
18.CDS-00rz9N:chr16:89919794	germline
19.CDS-00rz9N:chr17:28898692	germline. but likely driver.
20.CDS-00rz9N:chr17:4672736	germline

In [ ]:
parq = pd.read_parquet('gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/6b5fc00b-2b07-4b26-967e-1b4386a7094f/call-my_vcf_to_depmap/glob-233ae0abbeccb769e7426b0faeb6b946/f2351d30fada4391a5f07110744e543e-0.parquet')

In [ ]:
parq[(parq.chrom=="chr16")&(parq.pos>=780250)][["chrom", "variant_type", "variant_info","pos", "ref", "alt", "map_qual", "slippage", "strand_bias", "weak_evidence", "clustered_events", "base_qual", "likely_lof", "likely_driver", "is_coding", "germline", "pon", "popaf"]]

In [ ]:
res[(res.chrom=="chr16")&(res.pos>=780100)].sort_values(by=['chrom','pos'])

1. CDS-vaC7MY: ACH-000011: locat: 11:77934713 one wes rep. het-del in hg19. gene has been removed on hg38. and merged with another one at another location (78223667). still see the deletion there but was not picked to be in the maf as it was defined as germline by mutect2.


2. CDS-wA1WJ8: ACH-002467:chr22:39,966,847: one wes rep: het del in hhg19. can be seen in exon 1 of CACNA1I: loc on hg38 is 39,570,753 and the del can be seen at 39,570,842. It has no reason to be filtered and exist under chhr22:39,966,846 according to the chain file. this is because in one it is given as AT->A instead of a T->-. dropping cases like these doesn't change the overlap amount much..

3. CDS-uMD8jg:ACH-000038:chr4:188924749 only one wes rep: het SNP in hg19. can be seen in exon of ZFP42. in hg38 itss location is chr4:187,995,771. we can see the same G-A SNP in hg38 at location chr4:188,003,595. Doesn't exist in the mutect2 output. it is noted as germline and has no reasons to be whitelisted.

## looking at key mutations

In [ ]:
res[res.hugo_symbol=='JAK2'][[ 'af','gt', 'variant_info',
       'hugo_symbol','issues',
       'achilles_top_genes', 'ccle_deleterious', 'cosmic_hotspot', 'lof', 'driver', 'likely_driver',
       'transcript_likely_lof',
       'popaf', 'likely_gof', 'likely_lof', 'hess_driver',
       'sample_id']].iloc[:50]

In [ ]:
# we have more known mutations (only found not was CEBPA)
print('gene', 'new', 'old')
print('JAK2:',len(res[res.hugo_symbol=='JAK2']),len(oldmaf[oldmaf.Hugo_Symbol=='JAK2']))
print('EGFR:',len(res[res.hugo_symbol=='EGFR']),len(oldmaf[oldmaf.Hugo_Symbol=='EGFR']))
print('BRAF:',len(res[res.hugo_symbol=='BRAF']),len(oldmaf[oldmaf.Hugo_Symbol=='BRAF']))
print('VHL:',len(res[res.hugo_symbol=='VHL']),len(oldmaf[oldmaf.Hugo_Symbol=='VHL']))
print('FLT3:',len(res[res.hugo_symbol=='FLT3']),len(oldmaf[oldmaf.Hugo_Symbol=='FLT3']))
print('MEF2D:',len(res[res.hugo_symbol=='MEF2D']),len(oldmaf[oldmaf.Hugo_Symbol=='MEF2D']))
print('KRAS:',len(res[res.hugo_symbol=='KRAS']),len(oldmaf[oldmaf.Hugo_Symbol=='KRAS']))
print('BRCA2:',len(res[res.hugo_symbol=='BRCA2']),len(oldmaf[oldmaf.Hugo_Symbol=='BRCA2']))
print('TP53:',len(res[res.hugo_symbol=='TP53']),len(oldmaf[oldmaf.Hugo_Symbol=='TP53']))

In [ ]:
of these: what is the significance freq

In [ ]:
Achilles_gene_dependency = tc.get(name='depmap-a0ab', version=116, file='Achilles_gene_dependency')

In [ ]:
Achilles_gene_dependency.columns = [i.split(' (')[0] for i in Achilles_gene_dependency.columns]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

## KRAS

In [ ]:
datanew = Achilles_gene_dependency[['KRAS']].rename(columns={'KRAS':'KRAS_dep_score'})
datanew['KRAS_lof']=False
datanew['type']="new"
datanew.loc[list(set([rn[i] for i in res[(res.hugo_symbol=='KRAS') & ((res.likely_lof=="Y")|(res.ccle_deleterious=="Y"))].sample_id]) &set(datanew.index)), "KRAS_lof"]=True

dataold = Achilles_gene_dependency[['KRAS']].rename(columns={'KRAS':'KRAS_dep_score'})
dataold['KRAS_lof']=False
dataold['type']="old"
dataold.loc[list(set([rn[i] for i in oldmaf[(oldmaf.Hugo_Symbol=='KRAS') & (oldmaf.isDeleterious)].sample_id]) & set(dataold.index)), "KRAS_lof"]=True

In [ ]:
dataold.KRAS_lof.sum(), datanew.KRAS_lof.sum()

In [ ]:
sns.violinplot(data=datanew, x="KRAS_lof", y="KRAS_dep_score")
plt.show()

In [ ]:
sns.violinplot(data=dataold, x="KRAS_lof", y="KRAS_dep_score")
plt.show()

In [ ]:
stat, p_value = ttest_ind(datanew.loc[datanew['KRAS_lof'], 'KRAS_dep_score'].tolist(), datanew.loc[~datanew['KRAS_lof'], 'KRAS_dep_score'].tolist())
print(f"NEW:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}\n")

stat, p_value = ttest_ind(dataold.loc[dataold['KRAS_lof'], 'KRAS_dep_score'].tolist(), dataold.loc[~dataold['KRAS_lof'], 'KRAS_dep_score'].tolist())
print(f"OLD:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}")

## TP53

In [ ]:
datanew = Achilles_gene_dependency[['MDM2']].rename(columns={'MDM2':'MDM2_dep_score'})
datanew['TP53_lof']=False
datanew['type']="new"
datanew.loc[list(set([rn[i] for i in res[(res.hugo_symbol=='TP53') & ((res.likely_lof=="Y")|(res.ccle_deleterious=="Y"))].sample_id]) &set(datanew.index)), "TP53_lof"]=True

dataold = Achilles_gene_dependency[['MDM2']].rename(columns={'MDM2':'MDM2_dep_score'})
dataold['TP53_lof']=False
dataold['type']="old"
dataold.loc[list(set([rn[i] for i in oldmaf[(oldmaf.Hugo_Symbol=='TP53') & (oldmaf.isDeleterious)].sample_id]) & set(dataold.index)), "TP53_lof"]=True

In [ ]:
dataold.TP53_lof.sum(), datanew.TP53_lof.sum()

In [ ]:
sns.violinplot(data=datanew, x="TP53_lof", y="MDM2_dep_score")
plt.show()

In [ ]:
sns.violinplot(data=dataold, x="TP53_lof", y="MDM2_dep_score")
plt.show()

In [ ]:
stat, p_value = ttest_ind(datanew.loc[datanew['TP53_lof'], 'MDM2_dep_score'].tolist(), datanew.loc[~datanew['TP53_lof'], 'MDM2_dep_score'].tolist())
print(f"NEW:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}\n")

stat, p_value = ttest_ind(dataold.loc[dataold['TP53_lof'], 'MDM2_dep_score'].tolist(), dataold.loc[~dataold['TP53_lof'], 'MDM2_dep_score'].tolist())
print(f"OLD:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}")

## RB1 w/ E2F3 dep

In [ ]:
datanew = Achilles_gene_dependency[['E2F3']].rename(columns={'E2F3':'E2F3_dep_score'})
datanew['RB1_lof']=False
datanew['type']="new"
datanew.loc[list(set([rn[i] for i in res[(res.hugo_symbol=='RB1') & ((res.likely_lof=="Y") |(res.ccle_deleterious=="Y"))].sample_id]) &set(datanew.index)), "RB1_lof"]=True

dataold = Achilles_gene_dependency[['E2F3']].rename(columns={'E2F3':'E2F3_dep_score'})
dataold['RB1_lof']=False
dataold['type']="old"
dataold.loc[list(set([rn[i] for i in oldmaf[(oldmaf.Hugo_Symbol=='RB1') & (oldmaf.isDeleterious)].sample_id]) & set(dataold.index)), "RB1_lof"]=True

In [ ]:
dataold.RB1_lof.sum(), datanew.RB1_lof.sum()

In [ ]:
sns.violinplot(data=datanew, x="RB1_lof", y="E2F3_dep_score")
plt.show()

In [ ]:
sns.violinplot(data=dataold, x="RB1_lof", y="E2F3_dep_score")
plt.show()

In [ ]:
stat, p_value = ttest_ind(datanew.loc[datanew['RB1_lof'], 'E2F3_dep_score'].tolist(), datanew.loc[~datanew['RB1_lof'], 'E2F3_dep_score'].tolist())
print(f"NEW:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}\n")

stat, p_value = ttest_ind(dataold.loc[dataold['RB1_lof'], 'E2F3_dep_score'].tolist(), dataold.loc[~dataold['RB1_lof'], 'E2F3_dep_score'].tolist())
print(f"OLD:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}")

## BRAF w/ BRAF dep

In [ ]:
datanew = Achilles_gene_dependency[['BRAF']].rename(columns={'BRAF':'BRAF_dep_score'})
datanew['BRAF_lof']=False
datanew['type']="new"
datanew.loc[list(set([rn[i] for i in res[(res.hugo_symbol=='BRAF') & ((res.likely_lof=="Y") | (res.ccle_deleterious=='Y'))].sample_id]) &set(datanew.index)), "BRAF_lof"]=True

dataold = Achilles_gene_dependency[['BRAF']].rename(columns={'BRAF':'BRAF_dep_score'})
dataold['BRAF_lof']=False
dataold['type']="old"
dataold.loc[list(set([rn[i] for i in oldmaf[(oldmaf.Hugo_Symbol=='BRAF') & (oldmaf.isDeleterious)].sample_id]) & set(dataold.index)), "BRAF_lof"]=True

In [ ]:
dataold.BRAF_lof.sum(), datanew.BRAF_lof.sum()

In [ ]:
sns.violinplot(data=datanew, x="BRAF_lof", y="BRAF_dep_score")
plt.show()

In [ ]:
sns.violinplot(data=dataold, x="BRAF_lof", y="BRAF_dep_score")
plt.show()

In [ ]:
stat, p_value = ttest_ind(datanew.loc[datanew['BRAF_lof'], 'BRAF_dep_score'].tolist(), datanew.loc[~datanew['BRAF_lof'], 'BRAF_dep_score'].tolist())
print(f"NEW:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}\n")

stat, p_value = ttest_ind(dataold.loc[dataold['BRAF_lof'], 'BRAF_dep_score'].tolist(), dataold.loc[~dataold['BRAF_lof'], 'BRAF_dep_score'].tolist())
print(f"OLD:\nt-test: statistic={stat:.4f}, p-value={p_value:.4f}")

#### In the 4 cases tested, the new method is better in 3. Adding the old method's annotation makes the results worst in 2.

## plotting files

In [ ]:
converter["chr22"][39570841]

In [ ]:
res[res.cosmic_hotspot=='Y'].cosmic_overlapping_mutations

In [ ]:
res.columns

In [ ]:
res[(res.sample_id=='ACH-002467') & (res.chrom=='chr22')]

In [ ]:
pd.concat([res[res.locat.isin(set(res.locat) & set(CCLE_mutations.locat))].reset_index()[['alt', 'ref_count', 'alt_count', 'popaf', 'cosmic_hotspot', 'ccle_deleterious']], CCLE_mutations[CCLE_mutations.locat.isin(set(res.locat) & set(CCLE_mutations.locat))].reset_index()[['Alternate_Allele', 'isCOSMIChotspot', 'CGA_WES_AC', 'isDeleterious', 'locat']]],axis=1)

In [ ]:
res[res.likely_driver=='Y']

In [ ]:
CCLE_mutations

## LOF matrices

In [ ]:
from genepy.utils import helper as h

In [ ]:
parqfiles = {'CDS-00Nrzi': ['gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/6b5fc00b-2b07-4b26-967e-1b4386a7094f/call-my_vcf_to_depmap/glob-233ae0abbeccb769e7426b0faeb6b946/f2351d30fada4391a5f07110744e543e-0.parquet']}

In [ ]:
import requests
import json
from liftover import get_lifter
converter = get_lifter('hg19', 'hg38')

In [ ]:
converter["chr" + chrom_37][int(pos_37)]

In [ ]:
records[:40]

In [ ]:
set([i['type'] for i in records])

In [ ]:
civicdata = {}
page_url = 'https://civicdb.org/api/variants?count=500&page=1'
while page_url is not None:
    try:
        r = requests.get(page_url, timeout=5)
    except requests.exceptions.ConnectionError:
        msg = 'ERROR: Incomplete CIVIC data load'
        print(msg)
        break
    d = json.loads(r.text)
    records = d['records']
    page_url = d['_meta']['links']['next']
    for variant in records:
        print(varia)
        chrom_37 = variant['coordinates']['chromosome']
        pos_37 = variant['coordinates']['start']
        new_coords = converter["chr" + chrom_37][int(pos_37)]
        if len(new_coords) > 0:
            chrom_38 = new_coords[0][0].replace('chr','')
            pos_38 = new_coords[0][1]
        else:
            continue
        ref = variant['coordinates']['reference_bases']
        alt = variant['coordinates']['variant_bases']
        toks = [chrom_38, pos_38, ref, alt]
        if None not in toks:
            vkey = ':'.join(map(str, toks))
            civicdata[vkey] = variant
        else:
            continue   

In [ ]:
civicdata.keys()

In [ ]:
val.hgnc_symbol.unique()

In [ ]:
def makeMatrices(parqfiles, homin=0.95, **kwargs):
    val = h.generateGeneNames()#.hugo_symbol.unique()
    lof_mat = pd.DataFrame(index=val.hgnc_symbol.unique())
    driver_mat = pd.DataFrame(index=val.hgnc_symbol.unique())
    le = len(parqfiles)
    for j, (sam, parq) in enumerate(parqfiles.items()):
        h.showcount(i ,le)
        parq = pd.concat([pd.read_parquet(par) for par in parq])
        parq = parq[(parq.multiallelic!="Y")]
        parq = parq[(parq.af.astype(float)>=0.15)]
        lof = parq[parq.likely_lof=='Y']
        homlof = set(lof[lof['gt']=='1|1'].hugo_symbol)
        for dup in h.dups(lof.hugo_symbol):
            if lof[lof.hugo_symbol==dup]['af'].astype(float).sum() >= homin:
                homlof.add(dup)
        hetlof = set(lof.hugo_symbol)-homlof
        lof_mat.loc[homlof, sam] = '2'
        lof_mat.loc[hetlof, sam] = '1'
        # driver
        driver = parq[parq.civic_score!='']
        homdriv = set(driver[driver['gt']=='1|1'].hugo_symbol)
        for dup in h.dups(driver.hugo_symbol):
            if driver[driver.hugo_symbol==dup]['af'].astype(float).sum() >= homin:
                homdriv.add(dup)
        hetdriv = set(driver.hugo_symbol)-homdriv
        driver_mat.loc[homdriv, sam] = '2'
        driver_mat.loc[hetdriv, sam] = '1'
    lof_mat[lof_mat.isna().sum(1)<lof_mat.shape[1]]
    driver_mat[driver_mat.isna().sum(1)<driver_mat.shape[1]]
    return lof_mat, driver_mat

In [ ]:
parqfiles = {}
for k, v in samp[~samp.full_file.isna()].full_file.iteritems():
    if rn[k] in parqfiles:
        continue
    else:
        parqfiles[rn[k]] = v

In [ ]:
samp = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WGS_CN').get_samples()

In [ ]:
for k, v in samp[~samp.main_output.isna()].main_output.iteritems():
    if rn[k] in parqfiles:
        continue
    else:
        parqfiles[rn[k]] = v

In [ ]:
len(parqfiles)

In [ ]:
oldhugo = pd.read_csv('~/results.txt', sep='\t')

In [ ]:
remap = {b:a for a,b in oldhugo[~oldhugo['Previous symbol'].isna()].values}

In [ ]:
val = h.generateGeneNames()#.hugo_symbol.unique()
lof_mat = pd.DataFrame(index=val.hgnc_symbol.unique())
driver_mat = pd.DataFrame(index=val.hgnc_symbol.unique())
le = len(parqfiles)

lof_mat=lof_mat.rename(index=remap)
driver_mat=driver_mat.rename(index=remap)

In [ ]:
homin = 0.95

In [ ]:
j

In [ ]:
ind = set(lof_mat.index)

In [ ]:
failed = set()

In [ ]:
failed

In [ ]:
driver_mat = driver_mat.copy()
lof_mat = lof_mat.copy()

In [ ]:
parq['civic_score']

In [ ]:
list(parqfiles.keys())[1682]

In [ ]:
list(parqfiles.keys())[1683]

In [ ]:
list(parqfiles.keys())[1683]

In [ ]:
for j, (sam, parq) in enumerate(parqfiles.items()):
    h.showcount(j ,le)
    if j<1684:
        continue
    parq = pd.concat([pd.read_parquet(par) for par in parq])
    if len(set(parq.multiallelic)) > 2 or len(set(parq.likely_lof))>2:
        print(sam)
        continue
    parq = parq[(parq.multiallelic!="Y")]
    parq = parq[(parq.af.astype(float)>=0.15)]
    lof = parq[parq.likely_lof=='Y']
    homlof = set(lof[lof['gt']=='1|1'].hugo_symbol)
    for dup in h.dups(lof.hugo_symbol):
        if lof[lof.hugo_symbol==dup]['af'].astype(float).sum() >= homin:
            homlof.add(dup)
    hetlof = set(lof.hugo_symbol)-homlof
    homlof = [remap[i] if i in remap else i for i in homlof]
    hetlof = [remap[i] if i in remap else i for i in hetlof]
    failed = failed | set(homlof) - ind
    failed = failed | set(hetlof) - ind

    lof_mat.loc[list(set(homlof)-failed), sam] = '2'
    lof_mat.loc[list(set(hetlof)-failed), sam] = '1'
    # driver
    driver = parq[parq.civic_score!='']
    homdriv = set(driver[driver['gt']=='1|1'].hugo_symbol)
    for dup in h.dups(driver.hugo_symbol):
        if driver[driver.hugo_symbol==dup]['af'].astype(float).sum() >= homin:
            homdriv.add(dup)
    hetdriv = set(driver.hugo_symbol)-homdriv
    homdriv = [remap[i] if i in remap else i for i in homdriv]
    hetdriv = [remap[i] if i in remap else i for i in hetdriv]
    failed = failed | set(homdriv) - ind
    failed = failed | set(homdriv) - ind
    
    driver_mat.loc[list(set(homdriv)-failed), sam] = '2'
    driver_mat.loc[list(set(hetdriv)-failed), sam] = '1'
    if (j+1)%50==0:
        driver_mat = driver_mat.copy()
        lof_mat = lof_mat.copy()
lof_mat[lof_mat.isna().sum(1)<lof_mat.shape[1]]
driver_mat[driver_mat.isna().sum(1)<driver_mat.shape[1]]

In [ ]:
lof_mat[lof_mat.isna().sum(1)!=lof_mat.shape[1]].fillna('0').to_csv('lofmat.csv.gz')
driver_mat[driver_mat.isna().sum(1)!=driver_mat.shape[1]].fillna('0').to_csv('driver_mat.csv.gz')